# P-multigrid Example

In [1]:
# dependencies
using LFAToolkit
using LinearAlgebra
using Pkg
Pkg.activate("./")
Pkg.instantiate()
using Plots

 Activating environment at `~/Dev/LFAToolkit.jl/examples/jupyter/Project.toml`


In [12]:
# setup
finep = 9
midp5 = 5
midp3 = 3
coarsep = 2
dimension = 2

mesh = []
if dimension == 1
   mesh = Mesh1D(1.0)
elseif dimension == 2
   mesh = Mesh2D(1.0, 1.0)
end

mtofbasis5 = TensorH1LagrangeBasis(midp5, finep, dimension, lagrangequadrature=true)
mtofbasis3 = TensorH1LagrangeBasis(midp3, midp5, dimension, lagrangequadrature=true)
ctombasis = TensorH1LagrangeBasis(coarsep, midp3, dimension, lagrangequadrature=true)

# diffusion operators
finediffusion = GalleryOperator("diffusion", finep, finep, mesh)
middiffusion5 = GalleryOperator("diffusion", midp5, finep, mesh)
middiffusion3 = GalleryOperator("diffusion", midp3, finep, mesh)
coarsediffusion = GalleryOperator("diffusion", coarsep, finep, mesh)

# Jacobi smoother
finejacobi = Jacobi(finediffusion)
midjacobi5 = Jacobi(middiffusion5)
midjacobi3 = Jacobi(middiffusion3)

# p-multigrid preconditioner
midmultigrid3 = PMultigrid(middiffusion3, coarsediffusion, midjacobi3, [ctombasis])
midmultigrid5 = PMultigrid(middiffusion5, midmultigrid3, midjacobi5, [mtofbasis3])
multigrid = PMultigrid(finediffusion, midmultigrid5, finejacobi, [mtofbasis5])

p-multigrid preconditioner

In [16]:
# full operator symbols
numberruns = 170
maxeigenvalue = 0
θ_min = -π/2
θ_max = 3π/2

# compute and plot smoothing factor
# -- 1D --
if dimension == 1
    # setup
    ω = [0.576]
    v = [2, 2]
    maxeigenvalues = zeros(numberruns)

    # compute
    for i in 1:numberruns
        θ = [θ_min + (θ_max - θ_min)*i/numberruns]
        if abs(θ[1]  % 2π) % 2 >  π/32
            A = computesymbols(multigrid, ω, v, θ)
            eigenvalues = [abs(val) for val in eigvals(A)]
            maxeigenvalues[i] = max(eigenvalues...)
            maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i])
        end
    end

    # plot
    println("max eigenvalue: ", maxeigenvalue)
    xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
    plot(xrange, maxeigenvalues, title="P-Multigrid Symbols")
    ylims!(0.0, max(maxeigenvalues...))
# -- 2D --
elseif dimension == 2
    # setup
    ω = [0.795]
    v = [3, 3]
    maxeigenvalues = zeros(numberruns, numberruns)

    # compute
    for i in 1:numberruns, j in 1:numberruns
        θ = [
            θ_min + (θ_max - θ_min)*i/numberruns,
            θ_min + (θ_max - θ_min)*j/numberruns
        ]
        if sqrt(abs(θ[1] % 2π)^2 + abs(θ[2] % 2π)^2) > π/128
            A = computesymbols(multigrid, ω, v, θ)
            eigenvalues = [abs(val) for val in eigvals(A)]
            maxeigenvalues[i, j] = max(eigenvalues...)
            maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i, j])
        end
    end

    # plot
    println("max eigenvalue: ", maxeigenvalue)
    xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
    heatmap(
        xrange,
        xrange,
        maxeigenvalues,
        title="Spectral Radius of P-Multigrid Symbol",
        transpose=true,
        aspect_ratio=:equal
    )
    xlims!(θ_min/π, θ_max/π)
    ylims!(θ_min/π, θ_max/π)
    savefig("multigrid_spectral_radius_9_to_2_2d")
end

max eigenvalue: 0.27050859637097835
